In [1]:
#!pip install faker

In [2]:
from faker import Faker
import pandas as pd
import re
import random


In [3]:
faker = Faker()

In [4]:
data = pd.read_csv('Resumes_noPII.csv', encoding = 'utf-8')

In [5]:
data = pd.DataFrame(data)

In [6]:
data = data.drop('Resume_html', axis = 1)

In [7]:
data 

,ID,Resume_str,Category
0,1,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,2,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,3,HR DIRECTOR Summary Over 2...,HR
3,4,HR SPECIALIST Summary Dedica...,HR
4,5,HR MANAGER Skill Highlights ...,HR
...,...,...,...
2478,2479,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,AVIATION
2479,2480,"GOVERNMENT RELATIONS, COMMUNICATIONS ...",AVIATION
2480,2481,GEEK SQUAD AGENT Professional...,AVIATION
2481,2482,PROGRAM DIRECTOR / OFFICE MANAGER ...,AVIATION


In [8]:
data['text'] = data['Resume_str']

In [9]:
data = data.drop('Resume_str', axis = 1)

In [10]:
data

,ID,Category,text
0,1,HR,HR ADMINISTRATOR/MARKETING ASSOCIATE\...
1,2,HR,"HR SPECIALIST, US HR OPERATIONS ..."
2,3,HR,HR DIRECTOR Summary Over 2...
3,4,HR,HR SPECIALIST Summary Dedica...
4,5,HR,HR MANAGER Skill Highlights ...
...,...,...,...
2478,2479,AVIATION,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...
2479,2480,AVIATION,"GOVERNMENT RELATIONS, COMMUNICATIONS ..."
2480,2481,AVIATION,GEEK SQUAD AGENT Professional...
2481,2482,AVIATION,PROGRAM DIRECTOR / OFFICE MANAGER ...


In [11]:
def replace_city_state(text):
    # Enhanced pattern with negative lookahead to prevent matching skill lists
    city_state_pattern = r'(?<!Skills:)(?<!,)\b([A-Za-z]+(?:[\s-][A-Za-z]+)*),\s*([A-Z]{2})\b(?!(?:,\s*[A-Z]{2}\b)+)'   #negative lookback

    def replace_with_fake(match):
        city = faker.city()
        state = faker.state_abbr()
        return f" {city}, {state}  "

    new_text = re.sub(city_state_pattern, replace_with_fake, text)
    return new_text

In [12]:
#def replace_city_state(text):
 #   city_state_pattern = r'(\b\w+\b,\s+\b\w+\b)'  # Matches "City , State"
  #  new_text = re.sub(city_state_pattern, lambda x: f'{faker.city()}, {faker.state_abbr()}', text)
   # return new_text

In [13]:
data['text'] = data['text'].apply(replace_city_state)

In [14]:
def replace_placeholders(text):
    fake_company = faker.company()
    fake_city = faker.city()
    fake_state = faker.state_abbr()
    
    modified_text = re.sub(r'\s*Company Name\s*', lambda m: f' {faker.company()}  ', text, flags=re.IGNORECASE)

    # Updated pattern with word boundaries, whitespace, and space after replacement
    modified_text = re.sub(r'\b\s*City\s*,\s*State\s\s*\b', lambda m: f' {fake_city}, {fake_state}  ', modified_text, flags=re.IGNORECASE)

    return modified_text

In [15]:
data['text'] = data['text'].apply(replace_placeholders)

***

## Adding Names, Emails, and Phone Numbers


In [16]:
def format_name_email_phone(text):
    fake_first_name = faker.first_name()
    fake_last_name = faker.last_name()
    
    email_domains = [
        'gmail.com',
        "yahoo.com",
        "hotmail.com",
        "outlook.com",
        "example.com",
        "aol.com",
        "edu.com",
        "mail.com"
    ]
    
    selected_domain = random.choice(email_domains)
    
    fake_email = f'{fake_first_name.lower()}.{fake_last_name.lower()}@{selected_domain}'
    
    
    area_code = faker.random_int(min=100, max=999)
    central_office_code = faker.random_int(min=100, max=999)
    line_number = faker.random_int(min=1000, max=9999)
    fake_phone_number = f"({area_code}) {central_office_code}-{line_number}"
    
    
    formatted_string = f'{fake_first_name} {fake_last_name} \n\n{fake_email}\n\n{fake_phone_number}\n'
    
    return formatted_string + text

In [17]:
data['text'] = data['text'].apply(format_name_email_phone)

In [19]:
#data.to_csv('Resumes_with_PII_updated.csv', index=False, encoding='utf-8')